# 📚 Study Notes Image Classifier for ML Kit

This notebook guides you through creating a custom TensorFlow Lite model that classifies images as either "notes" or "others". The model uses a 2-class categorical architecture with softmax activation and will be fully compatible with ML Kit's Custom Image Labeling on Android.

## Model Architecture
- **Base model**: MobileNetV2 pretrained on ImageNet
- **Input shape**: (1, 224, 224, 3) — batch size 1, width & height 224, 3 RGB channels
- **Output layer**: Dense layer with 2 units (for classes "notes" and "others") with softmax activation
- **Output shape**: (1, 2) — tensor with class probabilities for the two classes

## Overview

1. Mount Google Drive & Install Dependencies
2. Load and Preprocess Dataset
3. Build & Train Model
4. Convert to TFLite
5. Add Required ML Kit Metadata
6. Verify Model & Test Classification

Let's get started!


## 1. Mount Google Drive & Install Dependencies


In [ ]:
# Mount Google Drive to access the dataset
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras import layers, models

!python --version
print(f"TensorFlow version: {tf.__version__}")

## 2. Load and Preprocess Dataset

First, let's set up the path to our training data.


In [ ]:
# Set the path to your dataset in Google Drive
# Update this path to match your Google Drive structure
data_dir = "/content/drive/MyDrive/training_data"

# Verify the directory structure
print(f"Classes: {os.listdir(data_dir)}")
print(f"Number of 'notes' images: {len(os.listdir(os.path.join(data_dir, 'notes')))}")
print(f"Number of 'others' images: {len(os.listdir(os.path.join(data_dir, 'others')))}")

Now, let's set up our data preprocessing and augmentation. This will help improve model performance and generalization.


In [ ]:
# Model settings
IMG_SIZE = 224  # Standard size for mobile models
BATCH_SIZE = 32
EPOCHS = 10

# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  # 20% for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

# Only rescaling for validation
validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",  # categorical classification for 2 classes
    subset="training",
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
)

Let's visualize some of our training images to check our data preprocessing:


In [ ]:
# Function to display images from our data generator
def show_batch(image_batch, label_batch):
    class_names = list(train_generator.class_indices.keys())
    plt.figure(figsize=(10, 10))
    for i in range(min(9, len(image_batch))):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(image_batch[i])
        label_idx = np.argmax(label_batch[i])
        plt.title(class_names[label_idx])
        plt.axis("off")
    plt.tight_layout()
    plt.show()


# Get a batch of training images
image_batch, label_batch = next(train_generator)
show_batch(image_batch, label_batch)

## 3. Build & Train Model

We'll use transfer learning with MobileNetV2, which is efficient and optimized for mobile applications.


In [ ]:
# Create a model using MobileNetV2 as the base model
def create_model():
    # Base model: MobileNetV2 (optimized for mobile)
    base_model = MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )

    # Freeze the base model layers
    base_model.trainable = False

    # Create the model - architecture with 2 output units and softmax
    model = models.Sequential(
        [
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.BatchNormalization(),
            layers.Dense(128, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(2, activation="softmax"),  # 2 units with softmax for "notes" and "others"
        ]
    )

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",  # categorical crossentropy for 2-class softmax
        metrics=["accuracy"],
    )

    return model


# Alternative model using EfficientNet
def create_efficient_model():
    base_model = EfficientNetB0(
        weights="imagenet",
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )

    base_model.trainable = False

    model = models.Sequential(
        [
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.BatchNormalization(),
            layers.Dense(128, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(2, activation="softmax"),  # 2 units with softmax
        ]
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


# Create the model
model = create_model()
# model = create_efficient_model()

# Display model summary
model.summary()

In [ ]:
# Use callbacks for better training
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True,
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.2,
        patience=2,
        min_lr=0.00001,
    ),
]

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
# Plot training history
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label="Training Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()

### Fine-tune the model

Let's unfreeze some of the base model layers and continue training with a lower learning rate for better accuracy.


In [ ]:
# Fine-tuning
def fine_tune_model():
    # Unfreeze the top layers of the model
    for layer in model.layers[0].layers[-30:]:  # Unfreeze last 30 layers of MobileNetV2
        layer.trainable = True

    # Recompile with a lower learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate
        loss="categorical_crossentropy",  # categorical crossentropy for 2-class softmax
        metrics=["accuracy"],
    )

    # Train for a few more epochs
    fine_tune_history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        epochs=5,  # Just a few more epochs
        callbacks=callbacks,
        verbose=1,
    )

    return fine_tune_history


# Run fine-tuning
fine_tune_history = fine_tune_model()

In [ ]:
# Plot fine-tuning results
ft_acc = fine_tune_history.history["accuracy"]
ft_val_acc = fine_tune_history.history["val_accuracy"]
ft_loss = fine_tune_history.history["loss"]
ft_val_loss = fine_tune_history.history["val_loss"]

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(ft_acc, label="Training Accuracy")
plt.plot(ft_val_acc, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(ft_loss, label="Training Loss")
plt.plot(ft_val_loss, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()

Now, let's save our trained model:


In [ ]:
# Create a directory for the SavedModel format
!mkdir -p saved_model

# Export the model
model.export('saved_model/')
print("Model saved in SavedModel format.")

## 4. Convert to TFLite

Next, we'll convert our SavedModel to TFLite format for mobile deployment, with optimization for best performance.


In [ ]:
# Basic conversion to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model/")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TFLite ops
    tf.lite.OpsSet.SELECT_TF_OPS     # enable TensorFlow ops
]
tflite_model = converter.convert()

# Save the TFLite model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print(f"TFLite model size: {len(tflite_model) / (1024 * 1024):.2f} MB")

In [ ]:
# Create a quantized model for better performance on mobile
def create_quantized_model():
    converter = tf.lite.TFLiteConverter.from_saved_model("saved_model/")
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # Add more quantization options for better mobile performance
    converter.target_spec.supported_ops = [
      tf.lite.OpsSet.TFLITE_BUILTINS_INT8,  # enable TFLite ops
      tf.lite.OpsSet.SELECT_TF_OPS     # enable TensorFlow ops
    ]

    # Keep input and output as float for compatibility with ML Kit
    converter.inference_input_type = tf.float32
    converter.inference_output_type = tf.float32

    # Representative dataset for calibration
    def representative_dataset_gen():
        # Use a few batches from the validation set for calibration
        for _ in range(10):
            img_batch, _ = next(validation_generator)
            yield [img_batch]

    converter.representative_dataset = representative_dataset_gen

    quantized_tflite_model = converter.convert()

    # Save the quantized model
    with open("model_quantized.tflite", "wb") as f:
        f.write(quantized_tflite_model)

    print(f"Quantized TFLite model size: {len(quantized_tflite_model) / (1024 * 1024):.2f} MB")
    return quantized_tflite_model


# Create a quantized model for better mobile performance
quantized_tflite_model = create_quantized_model()

## 5. Add Required ML Kit Metadata

Now we need to add the required metadata to make our model compatible with ML Kit's Custom Image Labeling.


In [ ]:
# Create a labels.txt file with our class names
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

with open('labels.txt', 'w') as f:
    for i in range(len(class_labels)):
        f.write(f"{class_labels[i]}\n")

print("Labels file created:")
!cat labels.txt

In [ ]:
%pip install tflite-support

In [ ]:
# Add metadata to the TFLite model
from tensorflow_lite_support.metadata.python import metadata
from tensorflow_lite_support.metadata import metadata_schema_py_generated as _metadata_fb
import flatbuffers
import os


# Function to add metadata to a model
def add_metadata(model_path, output_path):
    """Creates the metadata for a study notes image classifier."""

    # Creates model info.
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta.name = "Study Notes Classifier"
    model_meta.description = (
        "Categorical classifier that identifies whether an image contains "
        "study notes or other content. The model outputs probabilities "
        "for two classes: 'notes' and 'others' using softmax activation."
    )
    model_meta.version = "v1.0"
    model_meta.author = "Bit-Blazer"
    model_meta.license = (
        "MIT License. "
        "https://opensource.org/licenses/MIT"
    )

    # Creates input info.
    input_meta = _metadata_fb.TensorMetadataT()
    input_meta.name = "image"
    input_meta.description = (
        "Input image to be classified as 'notes' or 'others'. "
        "The expected image is {0} x {1}, with three channels (red, green, blue) "
        "per pixel. Each value in the tensor is normalized to the range [0, 1] "
        "using pixel_value / 255.0.".format(224, 224)
    )

    input_meta.content = _metadata_fb.ContentT()
    input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
    input_meta.content.contentProperties.colorSpace = _metadata_fb.ColorSpaceType.RGB
    input_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties
    )

    # Add normalization options for input preprocessing
    input_normalization = _metadata_fb.ProcessUnitT()
    input_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions
    )
    input_normalization.options = _metadata_fb.NormalizationOptionsT()
    input_normalization.options.mean = [0.0]
    input_normalization.options.std = [255.0]
    input_meta.processUnits = [input_normalization]

    # Add input statistics
    input_stats = _metadata_fb.StatsT()
    input_stats.max = [255.0]
    input_stats.min = [0.0]
    input_meta.stats = input_stats

    # Creates output info.
    output_meta = _metadata_fb.TensorMetadataT()
    output_meta.name = "probability"
    output_meta.description = (
        "Output probabilities for the two classes: 'notes' and 'others'. "
        "The tensor contains 2 values representing the softmax probabilities "
        "for each class. Index 0 corresponds to 'notes' class, "
        "index 1 corresponds to 'others' class."
    )

    output_meta.content = _metadata_fb.ContentT()
    output_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
    output_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.FeatureProperties
    )

    # Add score thresholding options (recommended by ML Kit docs)
    score_thresholding = _metadata_fb.ProcessUnitT()
    score_thresholding.optionsType = (
        _metadata_fb.ProcessUnitOptions.ScoreThresholdingOptions
    )
    score_thresholding.options = _metadata_fb.ScoreThresholdingOptionsT()
    score_thresholding.options.globalScoreThreshold = 0.5  # Default threshold for classification
    output_meta.processUnits = [score_thresholding]

    # Add output statistics
    output_stats = _metadata_fb.StatsT()
    output_stats.max = [1.0]
    output_stats.min = [0.0]
    output_meta.stats = output_stats

    # Add label file for ML Kit compatibility
    label_file = _metadata_fb.AssociatedFileT()
    label_file.name = os.path.basename("labels.txt")
    label_file.description = "Labels for study notes classification: notes and others."
    label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
    output_meta.associatedFiles = [label_file]

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_meta]
    subgraph.outputTensorMetadata = [output_meta]
    model_meta.subgraphMetadata = [subgraph]

    # Build and populate metadata
    b = flatbuffers.Builder(0)
    b.Finish(
        model_meta.Pack(b),
        metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER,
    )
    metadata_buf = b.Output()

    # Populate the model with metadata
    populator = metadata.MetadataPopulator.with_model_file(model_path)
    populator.load_metadata_buffer(metadata_buf)
    populator.load_associated_files(["labels.txt"])
    populator.populate()

    # Save the model with metadata to output path
    with open(model_path, "rb") as f:
        model_with_metadata = f.read()

    with open(output_path, "wb") as f:
        f.write(model_with_metadata)

    print(f"Model with detailed metadata saved to {output_path}")
    print(f"Metadata includes:")
    print(f"  - Model: {model_meta.name} v{model_meta.version}")
    print(f"  - Author: {model_meta.author}")
    print(f"  - License: MIT")
    print(f"  - Input: 224x224 RGB image with normalization [0,1]")
    print(f"  - Output: 2-class softmax probabilities for 'notes' and 'others'")
    print(f"  - Score Threshold: {score_thresholding.options.globalScoreThreshold} (recommended by ML Kit)")
    print(f"  - Labels: {label_file.name} attached")

    return model_with_metadata


# Add metadata to both models
model_with_metadata = add_metadata("model.tflite", "model_with_metadata.tflite")
quantized_model_with_metadata = add_metadata("model_quantized.tflite", "model_quantized_with_metadata.tflite")

## 6. Verify Model & Test Classification

Let's test our model on some sample images to make sure it's working correctly.


In [ ]:
# Function to load and preprocess an image for prediction
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(
        image_path, target_size=(IMG_SIZE, IMG_SIZE)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # Create batch dimension
    return img_array, img


# Function to make predictions using the SavedModel
def predict_with_model(image_path):
    img_array, img = preprocess_image(image_path)
    predictions = model.predict(img_array)

    # Get class indices mapping
    class_indices = train_generator.class_indices
    class_labels = {v: k for k, v in class_indices.items()}
    
    probabilities = predictions[0]
    predicted_class_idx = np.argmax(probabilities)
    predicted_class = class_labels[predicted_class_idx]
    confidence = float(probabilities[predicted_class_idx])

    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Prediction: {predicted_class} (Confidence: {confidence:.4f})")
    plt.axis("off")
    plt.show()

    # Display probabilities with correct class names
    prob_str = ", ".join([f"{class_labels[i]}={probabilities[i]:.4f}" for i in range(len(class_labels))])
    print(f"Class probabilities: {prob_str}")
    
    return predicted_class, confidence

In [ ]:
# Test on sample images from the validation set
def test_on_validation_images():
    # Test on notes images
    test_dir = os.path.join(data_dir, "notes")
    test_images = os.listdir(test_dir)[:5]  # Get first 5 images

    print("\n--- Testing on 'notes' images ---")
    for image_name in test_images:
        image_path = os.path.join(test_dir, image_name)
        print(f"Testing image: {image_name}")
        predicted_class, confidence = predict_with_model(image_path)
        print(f"Predicted class: {predicted_class}, Confidence: {confidence:.4f}")

    # Test on others images
    test_dir = os.path.join(data_dir, "others")
    test_images = os.listdir(test_dir)[:5]  # Get first 5 images

    print("\n--- Testing on 'others' images ---")
    for image_name in test_images:
        image_path = os.path.join(test_dir, image_name)
        print(f"Testing image: {image_name}")
        predicted_class, confidence = predict_with_model(image_path)
        print(f"Predicted class: {predicted_class}, Confidence: {confidence:.4f}")


# Run tests on validation images
test_on_validation_images()

## Test the TFLite Model

Now let's test our TFLite model with metadata to make sure it's working correctly.


In [ ]:
# Function to test the TFLite model
def test_tflite_model(tflite_model_path, image_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Check the type of the input tensor
    floating_model = input_details[0]["dtype"] == np.float32

    # Get input shape - NxHxWxC, H:1, W:2
    height = input_details[0]["shape"][1]
    width = input_details[0]["shape"][2]

    # Load an image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(height, width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)

    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if floating_model:
        img_array = img_array / 255.0  # Normalize to [0,1] as per our training

    # Add batch dimension
    input_data = np.expand_dims(img_array, axis=0).astype(input_details[0]["dtype"])

    # Set the tensor to point to the input data to be inferred
    interpreter.set_tensor(input_details[0]["index"], input_data)

    # Run the inference
    interpreter.invoke()

    # Extract the output
    output_data = interpreter.get_tensor(output_details[0]["index"])

    # Process the result - 2-class softmax output
    class_indices = train_generator.class_indices
    class_labels = {v: k for k, v in class_indices.items()}

    
    probabilities = output_data[0]
    predicted_class_idx = np.argmax(probabilities)
    predicted_class = class_labels[predicted_class_idx]
    confidence = float(probabilities[predicted_class_idx])

    print(f"  Result: {predicted_class} with confidence {confidence:.4f}")
    prob_str = ", ".join([f"{class_labels[i]}={probabilities[i]:.4f}" for i in range(len(class_labels))])
    print(f"  Class probabilities: {prob_str}")
    return predicted_class, confidence


# Test the TFLite models
def test_tflite_models():
    # Test normal TFLite model
    print("\n--- Testing TFLite model with metadata ---")
    tflite_model_path = "model_with_metadata.tflite"

    # Test on a few notes images
    test_dir = os.path.join(data_dir, "notes")
    test_images = os.listdir(test_dir)[:3]

    print("Testing on 'notes' images:")
    for image_name in test_images:
        image_path = os.path.join(test_dir, image_name)
        print(f"Image: {image_name}")
        test_tflite_model(tflite_model_path, image_path)

    # Test on a few others images
    test_dir = os.path.join(data_dir, "others")
    test_images = os.listdir(test_dir)[:3]

    print("\nTesting on 'others' images:")
    for image_name in test_images:
        image_path = os.path.join(test_dir, image_name)
        print(f"Image: {image_name}")
        test_tflite_model(tflite_model_path, image_path)

    # Test quantized model
    print("\n--- Testing Quantized TFLite model with metadata ---")
    quantized_model_path = "model_quantized_with_metadata.tflite"

    # Test on a few notes images
    test_dir = os.path.join(data_dir, "notes")
    test_images = os.listdir(test_dir)[:3]

    print("Testing on 'notes' images:")
    for image_name in test_images:
        image_path = os.path.join(test_dir, image_name)
        print(f"Image: {image_name}")
        test_tflite_model(quantized_model_path, image_path)

    # Test on a few others images
    test_dir = os.path.join(data_dir, "others")
    test_images = os.listdir(test_dir)[:3]

    print("\nTesting on 'others' images:")
    for image_name in test_images:
        image_path = os.path.join(test_dir, image_name)
        print(f"Image: {image_name}")
        test_tflite_model(quantized_model_path, image_path)


# Run tests on TFLite models
test_tflite_models()

## 7. Examine Model Metadata

Let's verify that our model has all the required metadata for ML Kit.


In [ ]:
# Display Metadata
# from tflite_support import metadata

from tensorflow_lite_support.metadata.python import metadata


# Function to display metadata of a model
def display_metadata(model_path):
    print(f"\n--- Metadata for {model_path} ---")

    # Print metadata
    displayer = metadata.MetadataDisplayer.with_model_file(model_path)
    metadata_json = displayer.get_metadata_json()
    print("Metadata JSON:")
    import json

    print(json.dumps(json.loads(metadata_json), indent=2))

    # Print associated files
    associated_files = displayer.get_packed_associated_file_list()
    print("\nAssociated files:")
    for file in associated_files:
        print(file)

    # Get label file content
    label_file_content = displayer.get_associated_file_buffer("labels.txt")
    print("\nLabels file content:")
    print(label_file_content.decode("utf-8"))


# Display metadata for both models
display_metadata("model_with_metadata.tflite")
display_metadata("model_quantized_with_metadata.tflite")

## 8. Save Models for Android ML Kit Integration

Let's save our TFLite models with metadata to use in our Android app with ML Kit.


In [ ]:
# Save the models to Google Drive
output_dir = '/content/drive/MyDrive/ml_kit_models'
!mkdir -p $output_dir

# Copy the model files
!cp model_with_metadata.tflite $output_dir/notes_classifier.tflite
!cp model_quantized_with_metadata.tflite $output_dir/notes_classifier_quantized.tflite

# Also save the labels file separately (for reference)
!cp labels.txt $output_dir/labels.txt

print(f"Models saved to {output_dir}/")
print(f"- Standard model: notes_classifier.tflite")
print(f"- Quantized model: notes_classifier_quantized.tflite")
print(f"- Labels file: labels.txt")

## Troubleshooting & Optimization Tips

### Common Issues:

1. **Poor Accuracy**:
   - Try collecting more training data
   - Use data augmentation (already implemented in this notebook)
   - Try different base models (MobileNetV2, EfficientNet)
   - Fine-tune more layers of the base model
2. **Model Size Issues**:

   - Use quantization (already shown in this notebook)
   - Try a smaller base model (MobileNetV2 instead of EfficientNet)
   - Use pruning techniques to remove unnecessary weights

3. **ML Kit Integration**:
   - Make sure the model has proper metadata
   - Verify model input/output tensors match ML Kit requirements
   - Check Android logs for specific error messages

### Optimizing Performance:

1. **Preprocessing**:
   - Ensure consistent preprocessing between training and inference
   - Consider adding normalization parameters to metadata
2. **Hardware Acceleration**:

   - ML Kit uses hardware acceleration when available
   - Can specify GPU/NNAPI delegates for specific use cases

3. **Further Model Optimization**:
   - Try dynamic range quantization or float16 quantization
   - Use TensorFlow Model Optimization Toolkit for advanced techniques


## Conclusion

You now have a fully functional TensorFlow Lite model with the specified architecture that can classify images using a 2-class categorical approach, ready for integration with ML Kit in your Android app. 

### Model Architecture Summary:
- **Base model**: MobileNetV2 pretrained on ImageNet
- **Input shape**: (1, 224, 224, 3) — batch size 1, width & height 224, 3 RGB channels
- **Output layer**: Dense layer with 2 units and softmax activation
- **Output shape**: (1, 2) — tensor with class probabilities for "notes" and "others"
- **Classes**: Determined by labels.txt file, mapped from folder structure

The model follows all the required specifications for ML Kit Custom Image Labeling, including proper input/output tensors and metadata.

This notebook has covered:

- Loading and preprocessing your dataset from Google Drive
- Building and training a categorical classifier using transfer learning
- Converting the model to TFLite format with proper 2-class softmax output
- Adding the required metadata for ML Kit compatibility
- Testing the model on sample images with correct class probability interpretation

The final model is saved in your Google Drive at `/content/drive/MyDrive/ml_kit_models/notes_classifier.tflite` and is ready to be used in your Android app!
